In [204]:
# import dependencies
import pandas as pd
from path import Path 
import numpy as np 
import os

In [205]:
# set path and load dataframe
path = Path('../data/raw_data/hmda_2017_ca_all-records_labels.csv')
df = pd.read_csv(path)

C:\Users\jsrit\AppData\Local\Temp\ipykernel_32696\4078387195.py:3: DtypeWarning: Columns (34,36,38,42,44,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [206]:
df.head()

,as_of_year,respondent_id,agency_name,agency_abbr,agency_code,loan_type_name,loan_type,property_type_name,property_type,loan_purpose_name,...,edit_status_name,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,application_date_indicator
0,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home improvement,...,NaN,NaN,NaN,4824.0,37.230000,75200.0,57.419998,818.0,1626.0,NaN
1,2017,0000146672,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,NaN,NaN,NaN,7404.0,57.520000,63200.0,116.010002,1215.0,1743.0,NaN
2,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,NaN,NaN,NaN,3372.0,33.189999,97400.0,141.740005,592.0,1105.0,NaN
3,2017,13-6131491,Department of Housing and Urban Development,HUD,7,Conventional,1,One-to-four family dwelling (other than manufa...,1,Refinancing,...,NaN,NaN,NaN,8787.0,65.129997,97400.0,97.269997,1463.0,2164.0,NaN
4,2017,33-0419992,Department of Housing and Urban Development,HUD,7,FHA-insured,2,One-to-four family dwelling (other than manufa...,1,Refinancing,...,NaN,NaN,NaN,5356.0,23.100000,75200.0,126.690002,1711.0,2102.0,NaN


In [207]:
df.isna().sum()

as_of_year                              0
respondent_id                           0
agency_name                             0
agency_abbr                             0
agency_code                             0
                                   ...   
hud_median_family_income             7461
tract_to_msamd_income                7461
number_of_owner_occupied_units       7461
number_of_1_to_4_family_units        7461
application_date_indicator        1714459
Length: 78, dtype: int64

In [208]:
# (in case) drop any rows with all null values 
df = df.dropna(how='all')


In [209]:
# df.isna().sum()
df_info = df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714459 entries, 0 to 1714458
Data columns (total 78 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   as_of_year                      int64  
 1   respondent_id                   object 
 2   agency_name                     object 
 3   agency_abbr                     object 
 4   agency_code                     int64  
 5   loan_type_name                  object 
 6   loan_type                       int64  
 7   property_type_name              object 
 8   property_type                   int64  
 9   loan_purpose_name               object 
 10  loan_purpose                    int64  
 11  owner_occupancy_name            object 
 12  owner_occupancy                 int64  
 13  loan_amount_000s                float64
 14  preapproval_name                object 
 15  preapproval                     int64  
 16  action_taken_name               object 
 17  action_taken               

In [210]:
# Check if respondent_id can serve as primary key
len(df['respondent_id'].unique())


# it cannot

1212

In [211]:
# find out value counts for a few columns
df['lien_status_name'].value_counts()

Secured by a first lien          1414092
Not applicable                    233447
Secured by a subordinate lien      47535
Not secured by a lien              19385
Name: lien_status_name, dtype: int64

In [212]:
df.co_applicant_sex_name.value_counts()

No co-applicant                                                                      829501
Female                                                                               539619
Male                                                                                 140865
Not applicable                                                                       132213
Information not provided by applicant in mail, Internet, or telephone application     72261
Name: co_applicant_sex_name, dtype: int64

In [213]:
df.purchaser_type_name.value_counts()

Loan was not originated or was not sold in calendar year covered by register    893403
Fannie Mae (FNMA)                                                               257246
Freddie Mac (FHLMC)                                                             171975
Ginnie Mae (GNMA)                                                               158456
Commercial bank, savings bank or savings association                             89764
Life insurance company, credit union, mortgage bank, or finance company          82080
Other type of purchaser                                                          31979
Private securitization                                                           17202
Affiliate institution                                                            12319
Farmer Mac (FAMC)                                                                   35
Name: purchaser_type_name, dtype: int64

In [214]:
df['hoepa_status_name'].value_counts()

Not a HOEPA loan    1713944
HOEPA loan              515
Name: hoepa_status_name, dtype: int64

In [215]:
print(df['denial_reason_name_1'].value_counts())
print(df.denial_reason_name_1.count())


Debt-to-income ratio                              49084
Credit history                                    28375
Credit application incomplete                     23034
Collateral                                        19886
Other                                             14627
Unverifiable information                           9060
Insufficient cash (downpayment, closing costs)     4858
Employment history                                 1531
Mortgage insurance denied                            67
Name: denial_reason_name_1, dtype: int64
150522


In [216]:
# Define column_value_counts function to determine how data is organized 
def column_value_counts(data):

    data = data.copy()

    column_list = []
    value_counts_dict = {}

    # loop through columns in dataframe 
    for column in data.columns:
        # check if 
        if not os.path.isdir(f'./column_value_counts/{column}'):
            os.makedirs(f'./column_value_counts/{column}')

        # assign value_counts for column to col_vc
        col_vc = data[column].value_counts()
        # make a dataframe, vc_df, from col_vc; reset index values, reset column names, and sort
        vc_df = pd.DataFrame(col_vc.reset_index().values, columns=['Value','Count']).sort_index(axis=0, ascending=True)
        # export column value counts df to csv in column_value_counts folder
        vc_df.to_csv(f'./column_value_counts/{column}/Value_Counts_{column}.csv', index=False)
        # set value_counts_dict {key: column, value: value counts df}
        value_counts_dict[column] = vc_df
        # append column name to column_list
        column_list.append(column)

    return column_list, value_counts_dict

In [217]:
column_list, value_counts_dict = column_value_counts(df)

In [218]:
value_counts_dict['loan_type_name']


,Value,Count
0,Conventional,1324862
1,FHA-insured,255304
2,VA-guaranteed,128964
3,FSA/RHS-guaranteed,5329


In [219]:
value_counts_dict['purchaser_type_name']


,Value,Count
0,Loan was not originated or was not sold in cal...,893403
1,Fannie Mae (FNMA),257246
2,Freddie Mac (FHLMC),171975
3,Ginnie Mae (GNMA),158456
4,"Commercial bank, savings bank or savings assoc...",89764
5,"Life insurance company, credit union, mortgage...",82080
6,Other type of purchaser,31979
7,Private securitization,17202
8,Affiliate institution,12319
9,Farmer Mac (FAMC),35


In [220]:
def compile_value_counts_df(vc_dict):
    # initialize compiled value counts dataframe
    comp_vc_df = pd.DataFrame()

    # iterate through value counts dictionary to extract value counts data for each column
    for key in vc_dict.keys():
        # set vc_df to the key's corresponding value (eg. column's value count dataframe)
        vc_df = vc_dict[key]
        vc_df.sort_values(by=['Count'], ascending=False)
        # create new Column by the same name, with key's name inputed
        vc_df['Column'] = f'{key}'
        # concatenate vc_df with comp_vc_df
        comp_vc_df = pd.concat([comp_vc_df, vc_df], ignore_index=True, axis=0)
    #return comp_vc_df
    return comp_vc_df

In [221]:
vc_df = compile_value_counts_df(vc_dict=value_counts_dict)

In [222]:
print(len(vc_df))
vc_df = vc_df[['Column','Value','Count']]
vc_df.to_csv('./columnar_value_counts.csv', index=False)
vc_df.head()


38477


,Column,Value,Count
0,as_of_year,2017,1714459
1,respondent_id,0000451965,153638
2,respondent_id,0000852218,72457
3,respondent_id,7197000003,59900
4,respondent_id,0000480228,52903


In [223]:
vc_df_sorted = vc_df.sort_values(by=['Count'], ascending=False)

vc_df_sorted.to_csv('./columnar_VC_high_low.csv', index=False)


In [224]:
# define get_unique_values_df function to make a dataframe of each column and its unique values
def get_unique_values_df(df):

    # initialize unique_dict and unique_values_list
    unique_values_list = []

    # copy dataframe
    df = df.copy()

    # iterate through columns
    for column in df.columns:
        # extract unique values
        unique_values = df[column].unique()
        # append unique values from each column to unique_values_list
        unique_values_list.append(unique_values)


    # make a dataframe of column name and unique values 
    values_df = pd.DataFrame({'Column Name':[col for col in df.columns], 'Unique_Count': [len(item) for item in unique_values_list], 'Unique_Values': unique_values_list}) 

    # return dataframe
    return values_df

In [225]:
unique_values_df = get_unique_values_df(df)
unique_values_df.to_csv('./column_data.csv', index=False)
